In [1]:
import torch

In [18]:
ids, activations = torch.load('/workspace/llm-progress-monitor/rollouts-test/activations/0.pt')

In [19]:
ids.shape

torch.Size([32])

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
tokenizer.decode(ids)

'<bos><start_of_turn>user\nWrite a short story about a cat and a dog.<end_of_turn>\n<start_of_turn>model\nJasper, a sleek black cat with ice-blue<end_of_turn>\n'

In [13]:
activations.shape

torch.Size([26, 33, 2304])

In [6]:
ids

tensor([     2,      2,    106,   1645,    108,   5559,    476,   3309,   3904,
          1105,    476,   4401,    578,    476,   5929, 235265,    107,    108,
           106,   2516,    108, 123635, 235269,    476,  69392,   2656,   4401,
           675,   8357, 235290,   8796,    107,    108])

In [1]:
import torch

In [2]:
torch.load('/workspace/llm-progress-monitor/rollouts-test/activations/0.pt')[1].shape

torch.Size([1, 449, 2560])

In [3]:
torch.load('/workspace/llm-progress-monitor/rollouts-test/activations/1.pt')[1]

tensor([[[-7.8800e+02,  5.0000e+00, -2.5250e+01,  ...,  7.1875e-01,
           5.1250e+00,  1.7344e+00],
         [ 3.5250e+01,  4.0625e+00, -7.2812e+00,  ..., -1.2988e-01,
           2.0508e-01,  1.7578e-01],
         [ 3.2500e+01,  3.7188e+00, -4.0312e+00,  ..., -7.2266e-01,
          -1.9336e-01,  1.0840e-01],
         ...,
         [ 2.0250e+01,  3.1250e+00,  2.5156e+00,  ..., -4.0625e-01,
           4.6289e-01,  3.1738e-02],
         [ 1.6125e+01,  3.4375e+00,  1.3047e+00,  ..., -9.7656e-03,
          -9.2773e-03, -1.2500e-01],
         [ 1.7250e+01,  4.0938e+00,  1.3594e+00,  ...,  2.9297e-02,
           1.1523e-01, -1.0498e-02]]], dtype=torch.bfloat16)

In [4]:
probe_weights = torch.load('/workspace/llm-progress-monitor/rollouts-test/probe_weights.pt')

In [5]:
probe_weights.shape

torch.Size([11, 2560])

In [9]:
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Convert to numpy for PCA
weights_np = probe_weights.float().cpu().detach().numpy()

# Fit PCA with 3 components
pca = PCA(n_components=3)
weights_pca = pca.fit_transform(weights_np)

# Create a DataFrame for plotly
df = pd.DataFrame({
    'PC1': weights_pca[:, 0],
    'PC2': weights_pca[:, 1], 
    'PC3': weights_pca[:, 2],
    'Point': [str(i) for i in range(len(weights_pca))]
})

# Create 3D scatter plot
fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3', 
                    text='Point',
                    title='Probe Weights in 3D PCA Space',
                    labels={
                        'PC1': f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)',
                        'PC2': f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)',
                        'PC3': f'PC3 ({pca.explained_variance_ratio_[2]:.2%} variance)'
                    })

# Update text position
fig.update_traces(textposition='middle right')

# Show the plot
fig.show()

print(f"Total variance explained by first 3 components: {pca.explained_variance_ratio_.sum():.2%}")


Total variance explained by first 3 components: 48.43%
